## Modulos

In [1]:
import numpy as np
import pandas as pd
from itertools import chain
from sklearn.linear_model import Perceptron, PassiveAggressiveClassifier, LogisticRegression
from sklearn.metrics import roc_auc_score
from sklearn.preprocessing import LabelEncoder, OneHotEncoder, StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer

from tqdm import tqdm
import warnings
warnings.filterwarnings('ignore')

## Funciones relevantes

In [2]:
def crear_individuo(num_genes=20):
    return [int(x>0.5) for x in np.random.uniform(size=num_genes)]

In [3]:
def seleccion(objetivo,pob,x_train,y_train,x_val,y_val,k = 3):
    pob = pd.DataFrame({'x':pob}).sample(k)
    pob['auc'] = pob['x'].map(lambda x:objetivo(x,x_train,y_train,x_val,y_val))
    pob.sort_values(by='auc',inplace=True)
    return pob.tail(1)['x'].values[0]

In [4]:
def combinacion(padre1,padre2,proba):
    #Los hijos son por defecto copias de los padres
    padre1 = [g for g in padre1]
    padre2 = [g for g in padre2]
    hijos = [padre1,padre2]
    if np.random.uniform()<proba:
        #Seleccionar un punto de combinacion que no esté
        #en los extremos
        i = np.random.choice(range(1,len(padre1)-1))
        hijos = padre1[:i]+padre2[i:],padre2[:i]+padre1[i:]
    return ["".join([str(x) for x in h]) for h in hijos]

In [5]:
def mutacion(individuo,proba=0.5):
    return "".join([str(1-int(g)) if r<proba else g for g,r in zip(individuo,np.random.uniform(size=len(individuo)))])

Aquí procedemos a interpretar la cadena binaria como los parametros del modelo a generar:
- Los primeros dos dígitos escogen uno de los 3 algoritmos (0-3)
- Se asigan los bits 2 y 3 para escoger la penalización que son L1,L2,None (0-3)
- Se toman los bits del 4-10 para el learning rate
    - Los dos primeros son para la parte entera (0-3)
    - Los 4 ultimos generan la parte decimal 
- Se toman los bits restantes para las máximas iteraciones, en el caso de ser de 20 genes nos da un número entre 0 y 511

En caso de querer mayor precisión sería aumentar el número de genes y aplear los bits del parametro

In [92]:
def decodificar(x):
    x="".join([str(s) for s in x])
    algoritmo= int(x[:2],2)
    if(algoritmo==3):
        algoritmo=2
    penalizacion=int(x[2:4],2)
    if(penalizacion==3):
        penalizacion=2
    if(algoritmo==2 and penalizacion==0):
        penalizacion+= np.random.randint(low=1, high=3)
    max_iter= int(x[10:],2)
    int_num = int(x[4:6], 2)
    frac_num = sum(int(bit) * 2 ** -(i + 1) for i, bit in enumerate(x[6:10]))
    lr= int_num+frac_num+.001
    return algoritmo,penalizacion,max_iter,lr

In [45]:
def objetivoAUC(x, x_train,y_train,x_val,y_val):
    hiper=decodificar(x)
    ls_penalty=["l1","l2", None]
    ls_models=[Perceptron(alpha=hiper[3], penalty=ls_penalty[hiper[1]], max_iter=hiper[2]),
           PassiveAggressiveClassifier(C=hiper[3],max_iter=hiper[2]),
           LogisticRegression(penalty=ls_penalty[hiper[1]],C=hiper[3],max_iter=hiper[2])]
    clf=ls_models[hiper[0]]
    try:
        clf.fit(x_train,y_train)
    except Exception as e:
        print(e)
        return 0
    y_proba= clf.decision_function(x_test)
    return roc_auc_score(y_test, y_proba)

Se recomienda 20 genes y mínimo deben de ser 11 dejando solo 1 o 0 iteraciones

In [17]:
def alg_genetico(objetivo,x_train,y_train,x_val,y_val,
                 n_genes=20,n_generaciones=100,tam_pob=100,proba_comb=0.9,proba_mut=1/20,tam_muestra=3):

    pob = [crear_individuo(num_genes=n_genes) for _ in range(tam_pob)]
    for gen in tqdm(range(n_generaciones)):
        aptos = [seleccion(objetivo,pob,x_train,y_train,x_val,y_val,k=tam_muestra) for _ in range(tam_pob)]
        hijos = [combinacion(*aptos[i:i+2],proba_comb) for i in range(0,tam_pob,2)]
        hijos = list(map(lambda x: mutacion(x,proba=proba_mut),chain(*hijos)))
        pob = hijos[:]
    pob = pd.DataFrame({'x':pob})
    pob['aptitud'] = pob['x'].map(lambda x: objetivo(x,x_train,y_train,x_val,y_val))
    pob.sort_values(by='aptitud',inplace=True)
    return pob.tail(1).values.tolist()[0]

In [36]:
def resultados(best):
    dc_model={0:"Perceptron", 1:"Pasivoo Agresivo", 2:"Regresión logistica", 3:"Regresión logistica"}
    dc_penaltys={0:"L1", 1:"L2", 2:"None", 3:"None"}
    ls_best_params=decodificar(best[0])
    resumen=f'''Model:{dc_model[ls_best_params[0]]})
    Regularizacion:{dc_penaltys[ls_best_params[1]]}
    lr:{ls_best_params[3]}
    Max_iters={ls_best_params[2]}
    AUC= {best[1]}
    '''
    print(resumen)

## Cargar datos

Se proporciona datos de clasificación para hacer la prueba, son los datos de Churn modeling se pueden encontrar en la siguiente url:  
https://www.kaggle.com/datasets/shubh0799/churn-modelling

In [10]:
data= pd.read_csv("Churn_Modelling.csv")

In [11]:
X= data.iloc[:,3:13].values
y= data.iloc[:,13].values
ls= ColumnTransformer([('Geography', OneHotEncoder(categories='auto'), [1])], remainder="passthrough")
X= ls.fit_transform(X)
X= X[:,1:]
ls= ColumnTransformer([('Gender', OneHotEncoder(categories='auto'), [3])], remainder="passthrough")
X= ls.fit_transform(X)
X= X[:,1:]

In [12]:
x_train,x_test,y_train,y_test= train_test_split(X,y, test_size=.20, random_state=0)

In [13]:
sx= StandardScaler()
x_train = sx.fit_transform(x_train)
x_test = sx.fit_transform(x_test)

## Test

In [95]:
best=alg_genetico(objetivoAUC,x_train,y_train,x_test,y_test,n_generaciones=75)

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 75/75 [04:38<00:00,  3.71s/it]


In [96]:
resultados(best)

Model:Regresión logistica)
    Regularizacion:L2
    lr:0.001
    Max_iters=677
    AUC= 0.7727311428460854
    


In [40]:
clf= LogisticRegression()

In [41]:
clf.fit(x_train,y_train)
y_proba= clf.decision_function(x_test)

In [42]:
roc_auc_score(y_test,y_proba)

0.7720283292697087